## 1.- Crear Base de Datos

* Crear con código SQL una base de datos llamada supermercado.
* Código Python con mysql connector que ejecute el SQL para borrar y generar la base de datos vacía

In [1]:
# Ejercicio 1 - Crear base de datos estudiantes
import mysql.connector as con
def crear_bd(sql):
    try:
       connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="ADMIN",
            database='supermercado'
        )
       cursor = connection.cursor()
       cursor.execute(sql)
       
    except con.Error as error:
      print(f"Ha ocurrido un error: {error}") 
      if connection:
         connection.rollback()
         return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

In [2]:
sql = "DROP DATABASE IF EXISTS supermercado; CREATE DATABASE IF NOT EXISTS supermercado;"

In [3]:
crear_bd(sql)

# Tablas SQL

* Tiendas

   * id_tienda (PRIMARY KEY)
   * nombre_tienda
   * direccion
   * ciudad


In [4]:
sql = "CREATE TABLE IF NOT EXISTS tiendas (id_tienda INT AUTO_INCREMENT PRIMARY KEY, nombre_tienda VARCHAR(100) NOT NULL, dirección VARCHAR(80), ciudad VARCHAR(50) NOT NULL);"

In [5]:
crear_bd(sql)

* Empleados
    * id_empleado (PRIMARY KEY)
    * nombre_empleado
    * puesto (ej.: Cajero, Gerente, Reponedor)
    * id_tienda (FOREIGN KEY que hace referencia a tiendas.id_tienda)

In [6]:
sql = "CREATE TABLE IF NOT EXISTS empleados (id_empleado INT AUTO_INCREMENT PRIMARY KEY, nombre_empleado VARCHAR(100) NOT NULL, puesto VARCHAR(60), id_tienda INT NOT NULL, FOREIGN KEY (id_tienda) REFERENCES tiendas(id_tienda) ON DELETE CASCADE);"

In [7]:
crear_bd(sql)

* Categorias
    * id_categoria (PRIMARY KEY)
    * nombre_categoria

In [8]:
sql = "CREATE TABLE IF NOT EXISTS categorias (id_categoria INT AUTO_INCREMENT PRIMARY KEY, nombre_categoria VARCHAR(60));"

In [9]:
crear_bd(sql)

* Productos
    * id_producto (PRIMARY KEY)
    * nombre_producto
    * precio
    * stock
    * id_categoria (FOREIGN KEY que hace referencia a categorias.id_categoria)

In [10]:
sql = "CREATE TABLE IF NOT EXISTS productos (id_producto INT AUTO_INCREMENT PRIMARY KEY, nombre_producto VARCHAR(70), precio FLOAT(8,2), stock INT, id_categoria INT NOT NULL, FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria));"

In [11]:
crear_bd(sql)

* Clientes
    * id_cliente (PRIMARY KEY)
    * first_name
    * last_name
    * email
    * codigo_postal

In [12]:
sql = "CREATE TABLE IF NOT EXISTS clientes (id_cliente INT AUTO_INCREMENT PRIMARY KEY, first_name VARCHAR(50), last_name VARCHAR(50), email VARCHAR(100), codigo_postal INT);"

In [13]:
crear_bd(sql)

* Ordenes
    * id_orden (PRIMARY KEY)
    * id_cliente (FOREIGN KEY que hace referencia a clientes.id_cliente)
    * id_empleado (FOREIGN KEY que hace referencia a empleados.id_empleado)
    * fecha_orden
    * metodo_pago (una enum que solo admita tres valores Tarjeta, Efectivo)

In [14]:
sql = "CREATE TABLE IF NOT EXISTS ordenes (id_orden INT AUTO_INCREMENT PRIMARY KEY, id_cliente INT, id_empleado INT, fecha_orden DATE, metodo_pago ENUM('Tarjeta', 'Efectivo') DEFAULT 'Efectivo', FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente), FOREIGN KEY (id_empleado) REFERENCES empleados(id_empleado));"

In [15]:
crear_bd(sql)

* Detalle_orden
    * id_detalle (PRIMARY KEY)
    * id_orden (FOREIGN KEY que hace referencia a ordenes.id_orden) NOT NULL
    * id_producto (FOREIGN KEY que hace referencia a productos.id_producto) NOT NULL
    * cantidad
    * precio_unitario: mismo precio que en la tabla producto
    * descuento (podría ser NULL si no se aplica)

In [16]:
sql = "CREATE TABLE IF NOT EXISTS detalle_orden (id_detalle INT AUTO_INCREMENT PRIMARY KEY, id_orden INT NOT NULL, id_producto INT NOT NULL, cantidad FLOAT(8,2), precio_unitario FLOAT(8,2), descuento TINYINT, FOREIGN KEY (id_orden) REFERENCES ordenes(id_orden), FOREIGN KEY (id_producto) REFERENCES productos(id_producto));"

In [17]:
crear_bd(sql)

# Tablas

## 2.- Generar datos demo desde Python

In [18]:
def insertar_dataframe(df, table_name):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="ADMIN",
            database='supermercado'
        )
        cursor = connection.cursor()
        
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        sql = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders});'
    
        rows = [tuple(row) for index, row in df.iterrows()] # lista de tuplas con los datos del dataframe

        cursor.executemany(sql, rows)
        
        connection.commit()
        return cursor.rowcount
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

* Tabla Tiendas

In [19]:
import pandas as pd
import random
import datetime

tiendas = ['Tienda Centro','Tienda Norte','Tienda Sur','Tienda Poniente','Tienda Este','Tienda Oeste','Central Tiendas']
tiendas

dirección = ['Avd.Centro,nº1','Calle Norte, nº2','Calle Sur, nº3', 'Avd. Poniente, nº4','Calle Este, nº5','Calle Oeste, nº6', 'Crta.Central, Km10']
   
dirección

# Ciudad

ciudad= ['Almería','Girona','Murcia','Barcelona','Orense','Gijón','Madrid' ] 
ciudad 

# Creamos un DataFrame 

df_tiendas = pd.DataFrame(zip(tiendas, dirección, ciudad),
             columns= ['nombre_tienda', 'dirección', 'ciudad']) # El comando Zip empaqueta todas las variante hemos hecho | y con columns damos nombres a las columnas

# Añadir columna Pk:

df_tiendas = df_tiendas.reset_index().rename({'index': 'id_tienda'}, axis=1)
df_tiendas['id_tienda'] = df_tiendas['id_tienda'] + 1
df_tiendas




,id_tienda,nombre_tienda,dirección,ciudad
0,1,Tienda Centro,"Avd.Centro,nº1",Almería
1,2,Tienda Norte,"Calle Norte, nº2",Girona
2,3,Tienda Sur,"Calle Sur, nº3",Murcia
3,4,Tienda Poniente,"Avd. Poniente, nº4",Barcelona
4,5,Tienda Este,"Calle Este, nº5",Orense
5,6,Tienda Oeste,"Calle Oeste, nº6",Gijón
6,7,Central Tiendas,"Crta.Central, Km10",Madrid


In [20]:
insertar_dataframe(df_tiendas, 'tiendas')

7

* Tabla Empleados

In [21]:
# Creamos lista de empleados | Nombre,Apellidos,Función
empleados = list()
nombres = ['Alejandro', 'María', 'Carlos', 'Lucía', 'Fernando', 'Isabel', 'Javier', 'Ana', 'Miguel', 'Sofía', 'José', 'Clara', 'Luis', 'Elena', 'Daniel', 'Marta', 'Andrés', 'Laura', 'Raúl', 'Paula', 'Manuel', 'Sara', 'Pablo', 'Julia', 'Sergio', 'Valeria', 'Tomás', 'Adriana', 'Eduardo', 'Camila', 'Francisco', 'Diana', 'Ricardo', 'Patricia', 'Ángel', 'Carla', 'Diego', 'Irene', 'Samuel', 'Eva', 'David', 'Andrea', 'Hugo', 'Verónica', 'Cristian', 'Gabriela', 'Mario', 'Natalia', 'Jorge', 'Lorena', 'Rubén', 'Carmen']
apellidos = ['García', 'Martínez', 'López', 'Sánchez', 'Pérez', 'González', 'Rodríguez', 'Fernández', 'Hernández', 'Jiménez', 'Ruiz', 'Díaz', 'Moreno', 'Álvarez', 'Romero', 'Torres', 'Vázquez', 'Domínguez', 'Ramos', 'Ramírez', 'Castro', 'Ortiz', 'Rubio', 'Molina', 'Delgado', 'Cruz', 'Flores', 'Navarro', 'Cabrera', 'Campos', 'Peña', 'Aguilar', 'Santos', 'Reyes', 'Méndez', 'Medina', 'Paredes', 'Serrano', 'Cortés', 'Carrillo', 'Ortiz', 'Velázquez', 'Chávez', 'Guzmán', 'Salinas', 'Esquivel', 'Lara', 'Fuentes', 'Montes']
función = ["Cajero", "Reponedor", "Encargado de sección", "Gerente", "Personal de limpieza", "Panadero", "Carnicero", "Pescadero", "Frutero", "Charcutero", "Jefe de turno", "Operador logístico", "Atención al cliente", "Auxiliar de caja", "Encargado de seguridad","Responsable de zona","Jefe de compras","Encargada carnicería","encargado pescadería","Jefe de Limpieza"]
for tienda in df_tiendas['id_tienda']:
    for empleado in range(20):
        nombre_empleado = (random.choice(nombres) + ' ' + random.choice(apellidos))
        puesto = función[empleado]
        id_tienda = tienda
        empleados.append([nombre_empleado, puesto, id_tienda])
        
# Creamos un DataFrame         
df_empleados = pd.DataFrame(empleados, columns=['nombre_empleado', 'puesto', 'id_tienda'])
        
df_empleados = df_empleados.reset_index().rename({'index': 'id_empleado'}, axis=1)
df_empleados['id_empleado'] = df_empleados['id_empleado'] + 1
df_empleados






,id_empleado,nombre_empleado,puesto,id_tienda
0,1,Mario Guzmán,Cajero,1
1,2,Irene Ortiz,Reponedor,1
2,3,Adriana Cortés,Encargado de sección,1
3,4,David Lara,Gerente,1
4,5,Laura Aguilar,Personal de limpieza,1
...,...,...,...,...
135,136,Carlos Delgado,Responsable de zona,7
136,137,David Castro,Jefe de compras,7
137,138,Daniel Guzmán,Encargada carnicería,7
138,139,Raúl Sánchez,encargado pescadería,7


In [22]:
insertar_dataframe(df_empleados, "empleados")

140

* Tabla Categorías

In [23]:
categorias = ['Lácteos','Pescados','Carnes','Verduras','Frutas','Bollería y Pan','Refrescos','Vinos','Alcohol','Congelados']

df_categorias= pd.DataFrame(categorias,columns= ['nombre_categoria'])

# Añadir columna pk:

df_categorias = df_categorias.reset_index().rename({'index': 'id_categoria'}, axis=1)
df_categorias['id_categoria'] = df_categorias['id_categoria'] + 1
df_categorias



,id_categoria,nombre_categoria
0,1,Lácteos
1,2,Pescados
2,3,Carnes
3,4,Verduras
4,5,Frutas
5,6,Bollería y Pan
6,7,Refrescos
7,8,Vinos
8,9,Alcohol
9,10,Congelados


In [24]:
insertar_dataframe(df_categorias, 'categorias')

10

* Productos

    * id_producto: valores enteros consecutivos (1, 2, 3...).
    * nombre_producto: nombres como “Leche Entera”, “Manzana Roja”, “Refresco de Cola”, etc.
    * precio: valores DECIMAL entre 0.50 y 50.00, por ejemplo.
    * stock: valores INT entre 0 y 500 (o el rango que quieras).
    * id_categoria: debe hacer referencia a las categorías que hayas definido (1, 2, 3, etc.).



In [25]:
# Listado de productos

# generar columna id_producto
nombres = ["Leche", "Yogur", "Queso", "Mantequilla", "Pollo", "Carne de res", "Cerdo", "Chuletas", "Manzana", "Banana", "Naranja", "Uva", "Zanahoria", "Tomate", "Lechuga", "Cebolla", "Agua", "Jugo de naranja", "Refresco", "Cerveza", "Papas fritas", "Chocolate", "Galletas", "Chicles", "Velas", "Platos desechables", "Bolsas de basura", "Pilas", "Detergente", "Jabón líquido", "Esponjas", "Desinfectante", "Champú", "Colonia", "Crema hidratante", "Desodorante", "Tierra para macetas", "Fertilizante", "Semillas", "Regadera"]
precios, lista_categorias, stock = list(), list(), list()
cantidad = len(nombres)
for _ in range(cantidad):
    precios.append(random.uniform(0.50, 10.50))
    stock.append(random.randint(0, 500))
for categ in df_categorias['id_categoria']:
    for i in range(4):
        lista_categorias.append(categ)
lista_categorias        
df_productos = pd.DataFrame(zip(nombres, precios, stock, lista_categorias), columns=['nombre_producto', 'precio', 'stock', 'id_categoria'])    
df_productos = df_productos.reset_index().rename({'index': 'id_producto'}, axis=1)
df_productos['id_producto'] = df_productos['id_producto'] + 1
df_productos

,id_producto,nombre_producto,precio,stock,id_categoria
0,1,Leche,8.721590,466,1
1,2,Yogur,10.416025,473,1
2,3,Queso,1.239585,392,1
3,4,Mantequilla,9.487563,381,1
4,5,Pollo,2.292598,46,2
5,6,Carne de res,1.938636,76,2
6,7,Cerdo,4.912155,18,2
7,8,Chuletas,2.185210,353,2
8,9,Manzana,8.674072,240,3
9,10,Banana,2.389311,156,3


In [26]:
insertar_dataframe(df_productos, 'productos')

40

* Clientes

    * id_cliente: valores enteros consecutivos (1, 2, 3...).
    * nombre_cliente: nombres y apellidos ficticios (p. ej. “Carlos López”, “María Gil”).
    * email: podrías generar correos ficticios (p. ej. “carlos.lopez@test.com”).
    * telefono: número de 9 o 10 dígitos (dependiendo del país).
    * direccion: calles y números ficticios (p. ej. “Av. Siempre Viva 742”).


In [27]:
# generar columna cliente
nombre_cliente, apellido_cliente, email, postal, cliente_completo = list(), list(), list(), list(), list()
calles = ("Av. Libertad", "Calle Mayor", "Paseo de la Reforma", "Calle Real", "Av. de la Constitución", "Calle del Sol", "Calle de la Luna", "Calle los Pinos", "Calle de la Paz", "Calle Olivo", "Calle Cedro", "Calle Nogal", "Av. de las Flores", "Paseo del Prado", "Calle del Río", "Av. Central", "Calle Primavera", "Calle Otoño", "Calle Invierno", "Calle Verano", "Calle Granada", "Calle Sevilla", "Calle Córdoba", "Calle Málaga", "Calle Valencia", "Calle Zaragoza", "Calle Alicante", "Av. de los Ángeles", "Calle del Carmen", "Calle San Miguel", "Calle San Juan", "Calle del Pilar", "Calle Santa Teresa", "Calle San Francisco", "Calle de la Sierra", "Calle del Mar", "Av. del Norte", "Av. del Sur", "Calle del Este", "Calle del Oeste", "Calle Diamante", "Calle Esmeralda", "Calle Rubí", "Calle Zafiro", "Calle Ámbar", "Calle Turquesa", "Calle Topacio", "Calle Perla", "Calle Horizonte", "Calle Amanecer")
for _ in range(2000):
    nom = random.choice(nombres)
    nombre_cliente.append(nom)
    ape = random.choice(apellidos)
    apellido_cliente.append(ape)
    email.append((f'{nom}.{ape}@mail.com'))
    postal.append(random.randint(0000, 11900))
    
df_clientes = pd.DataFrame(zip(nombre_cliente, apellido_cliente, email, postal), columns=['first_name', 'last_name', 'email', 'codigo_postal']) 
df_clientes = df_clientes.reset_index().rename({'index': 'id_cliente'}, axis=1)
df_clientes['id_cliente'] = df_clientes['id_cliente'] + 1
df_clientes

,id_cliente,first_name,last_name,email,codigo_postal
0,1,Fertilizante,Flores,Fertilizante.Flores@mail.com,3032
1,2,Colonia,Guzmán,Colonia.Guzmán@mail.com,10242
2,3,Papas fritas,Cruz,Papas fritas.Cruz@mail.com,66
3,4,Fertilizante,Fuentes,Fertilizante.Fuentes@mail.com,5890
4,5,Papas fritas,Moreno,Papas fritas.Moreno@mail.com,3774
...,...,...,...,...,...
1995,1996,Chocolate,Ramos,Chocolate.Ramos@mail.com,399
1996,1997,Colonia,Medina,Colonia.Medina@mail.com,5431
1997,1998,Tierra para macetas,Navarro,Tierra para macetas.Navarro@mail.com,6542
1998,1999,Leche,Ramírez,Leche.Ramírez@mail.com,7242


In [28]:
insertar_dataframe(df_clientes, 'clientes')

2000

* Orden
    * id_orden: Valores enteros consecutivos
    * id_cliente: Referencias de los Ids existentes en la tabla Clientes
    * id_empleado: Refeferencias de los Ids existentes en la tabla Empleados
    * fecha_orden: Generar fechas aleatorias o secuenciales
    * metodo_pago: Escoger entre tarjeta o efectivo

In [29]:
# generar columna id_orden
from datetime import timedelta


cliente, emplea, fecha, metodo = list(), list(), list(), list()
fechas = [datetime.datetime(year=2024, month=1, day=1) + timedelta(days=numero) for numero in range(395)]
for _ in range(10000):
    cliente.append(random.choice(df_clientes['id_cliente']))
    emplea.append(random.choice(df_empleados['id_empleado']))
    fecha.append(random.choice(fechas))
    metodo.append(random.choices(['Tarjeta', 'Efectivo'], weights=[0.65, 0.35])[0])

fecha.sort()
df_ordenes = pd.DataFrame(zip(cliente, emplea, fecha, metodo), columns=['id_cliente', 'id_empleado', 'fecha_orden', 'metodo_pago'])

df_ordenes.sort_values('fecha_orden', inplace=True)
df_ordenes = df_ordenes.reset_index().rename({'index': 'id_orden'}, axis=1)
df_ordenes['id_orden'] = df_ordenes['id_orden'] + 1
df_ordenes.sort_values('id_orden', inplace=True)
df_ordenes.head(50)

,id_orden,id_cliente,id_empleado,fecha_orden,metodo_pago
0,1,1757,5,2024-01-01,Tarjeta
18,2,1358,35,2024-01-01,Tarjeta
19,3,656,124,2024-01-01,Efectivo
25,4,120,30,2024-01-01,Tarjeta
21,5,290,94,2024-01-01,Tarjeta
22,6,1522,44,2024-01-01,Efectivo
23,7,610,5,2024-01-01,Tarjeta
24,8,409,70,2024-01-01,Tarjeta
31,9,1855,12,2024-01-01,Tarjeta
26,10,1022,117,2024-01-01,Efectivo


In [30]:
insertar_dataframe(df_ordenes, 'ordenes')

10000

* Detalles de ordenes
    * id_detalle: Valores enterors consecutivos
    * id_orden: Referencia al Id de alguna orden válida
    * id_producto: Referencia al ID
    * Cantidad: Valores entre 1 y 20 
    * Precio_unitario: Usando en mismo precio está en la tabla productos 
    * Descuento: Valores Decimales bajos

In [31]:
# generar columna id_detalle

orden_completa = list()
productos = df_productos['id_producto']
orden_20 = df_ordenes['id_orden']

for orden in df_ordenes['id_orden']:
    ordenes = orden
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor, 2), round(descuento, 2)])   
    
for orden in range(20000):
    ordenes = random.choice(orden_20)
    producto = (int(random.choice(productos)))
    cant = random.randint(1, 20)
    valor = float(df_productos['precio'][df_productos['id_producto'] == producto].iloc[0])
    descuento = random.uniform(0, 0.25)
    orden_completa.append([ordenes, producto, cant, round(valor,2), round(descuento, 2)])   
    
df_detalle_orden = pd.DataFrame(orden_completa, columns=['id_orden', 'id_producto', 'cantidad', 'precio_unitario', 'descuento'])

df_detalle_orden = df_detalle_orden.reset_index().rename({'index': 'id_detalle'}, axis=1)
df_detalle_orden['id_detalle'] = df_detalle_orden['id_detalle'] + 1
df_detalle_orden

,id_detalle,id_orden,id_producto,cantidad,precio_unitario,descuento
0,1,1,12,11,1.77,0.17
1,2,2,13,15,8.79,0.22
2,3,3,28,13,8.45,0.15
3,4,4,8,14,2.19,0.01
4,5,5,35,3,10.18,0.05
...,...,...,...,...,...,...
29995,29996,5283,21,10,2.75,0.16
29996,29997,4004,27,12,3.21,0.01
29997,29998,8316,3,20,1.24,0.21
29998,29999,3784,13,15,8.79,0.11


In [32]:
insertar_dataframe(df_detalle_orden, 'detalle_orden')

30000

## 3.- Consultas SQL

In [33]:
def consultas(sql):
    try:
        connection = con.connect(
            host="localhost",
            port="3306",
            user="root",
            password="ADMIN",
            database='supermercado'
        )
        cursor = connection.cursor()
        cursor.execute(sql)
        return cursor.fetchall()
    except con.Error as error:
        print(f"Ha ocurrido un error: {error}")
        if connection:
            connection.rollback()
        return 0
    finally:
        if cursor: 
            cursor.close()
        if connection:
            connection.close()

* 1.-Listado de órdenes con detalles de cliente y empleado

    * Muestra el ID de la orden, la fecha, el nombre del cliente, el nombre del empleado que atendió la compra y el método de pago.
    * Utiliza un JOIN entre las tablas ordenes, clientes y empleados

In [34]:
sql = ('SELECT o.id_orden, o.fecha_orden, c.first_name, c.last_name, e.nombre_empleado, o.metodo_pago FROM ordenes o JOIN clientes c ON c.id_cliente = o.id_cliente JOIN empleados e ON e.id_empleado = o.id_empleado ORDER BY o.id_orden;') 
consultas(sql)

[(1,
  datetime.date(2024, 1, 1),
  'Colonia',
  'Martínez',
  'Laura Aguilar',
  'Tarjeta'),
 (2,
  datetime.date(2024, 1, 1),
  'Semillas',
  'Cruz',
  'Paula Martínez',
  'Tarjeta'),
 (3,
  datetime.date(2024, 1, 1),
  'Uva',
  'González',
  'Gabriela Cruz',
  'Efectivo'),
 (4,
  datetime.date(2024, 1, 1),
  'Detergente',
  'Martínez',
  'Fernando Pérez',
  'Tarjeta'),
 (5,
  datetime.date(2024, 1, 1),
  'Chocolate',
  'Fernández',
  'Carlos González',
  'Tarjeta'),
 (6, datetime.date(2024, 1, 1), 'Uva', 'Cabrera', 'María García', 'Efectivo'),
 (7,
  datetime.date(2024, 1, 1),
  'Platos desechables',
  'Castro',
  'Laura Aguilar',
  'Tarjeta'),
 (8,
  datetime.date(2024, 1, 1),
  'Mantequilla',
  'Sánchez',
  'Carlos Castro',
  'Tarjeta'),
 (9,
  datetime.date(2024, 1, 1),
  'Manzana',
  'Fernández',
  'Cristian Reyes',
  'Tarjeta'),
 (10, datetime.date(2024, 1, 1), 'Uva', 'Peña', 'Mario Serrano', 'Efectivo'),
 (11,
  datetime.date(2024, 1, 1),
  'Cerveza',
  'Molina',
  'David Peña

* 2.- Productos con stock bajo

    * Filtra aquellos productos cuyo stock sea menor a 10.
    * Muestra nombre del producto, categoría y stock.

In [35]:
sql = ('SELECT p.nombre_producto, c.nombre_categoria, p.stock FROM productos p JOIN categorias c ON p.id_categoria = c.id_categoria WHERE p.stock < 10;')
consultas(sql)

[]

* 3.-Ventas totales por categoría

    * Muestra el nombre de la categoría y la suma total de las ventas (ej.: multiplicando cantidad * precio_unitario) para cada categoría.
    * Realiza el JOIN con detalle_orden, productos y categorias.
    * Utiliza agrupación (GROUP BY).

In [36]:
sql = ('SELECT c.nombre_categoria as categoria, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as ventas FROM categorias c JOIN productos p ON p.id_categoria = c.id_categoria JOIN detalle_orden d ON d.id_producto = p.id_producto GROUP BY c.id_categoria;')
consultas(sql)

[('Carnes', 133396.9),
 ('Verduras', 217190.08),
 ('Refrescos', 206642.67),
 ('Pescados', 90440.2),
 ('Alcohol', 200158.52),
 ('Congelados', 225867.91),
 ('Lácteos', 230800.17),
 ('Frutas', 172432.35),
 ('Bollería y Pan', 146404.7),
 ('Vinos', 254856.44)]

* 4.-Clientes con mayores gastos acumulados


    * Muestra el nombre del cliente y el monto total que ha gastado (suma de todas sus órdenes).
    * Asegúrate de tener en cuenta posibles descuentos (descuento) si se ha definido. Por ejemplo, la fórmula podría ser (cantidad * precio_unitario) - descuento.
    * Ordena el resultado de mayor a menor gasto acumulado.

In [37]:
sql = ('SELECT c.first_name as nombre, c.last_name as apellido, sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) as compras FROM clientes c JOIN ordenes o ON c.id_cliente = o.id_cliente JOIN detalle_orden d ON d.id_orden = o.id_orden GROUP BY c.id_cliente ORDER BY sum((d.precio_unitario * ( 1 - d.descuento)) * d.cantidad) DESC;')
consultas(sql)

[('Cerdo', 'Jiménez', 2979.22),
 ('Pilas', 'Domínguez', 2934.05),
 ('Tierra para macetas', 'Rubio', 2860.61),
 ('Chocolate', 'Rodríguez', 2781.4),
 ('Mantequilla', 'Domínguez', 2707.46),
 ('Fertilizante', 'Ortiz', 2694.12),
 ('Galletas', 'Cortés', 2658.16),
 ('Tierra para macetas', 'Navarro', 2597.72),
 ('Banana', 'García', 2587.74),
 ('Semillas', 'Medina', 2577.9),
 ('Uva', 'López', 2570.9),
 ('Detergente', 'Peña', 2555.82),
 ('Galletas', 'García', 2550.04),
 ('Chicles', 'Flores', 2548.3),
 ('Carne de res', 'Ortiz', 2542.73),
 ('Tomate', 'Castro', 2524.14),
 ('Detergente', 'Martínez', 2496.52),
 ('Chuletas', 'Domínguez', 2495.05),
 ('Refresco', 'López', 2481.41),
 ('Platos desechables', 'Vázquez', 2457.98),
 ('Desodorante', 'Guzmán', 2431.24),
 ('Carne de res', 'Medina', 2424.46),
 ('Cerveza', 'Rodríguez', 2403.49),
 ('Chuletas', 'González', 2402.6),
 ('Champú', 'Montes', 2400.01),
 ('Crema hidratante', 'Domínguez', 2396.32),
 ('Naranja', 'Medina', 2390.5),
 ('Uva', 'González', 2386.1

* 5.-Empleados y número de órdenes gestionadas

    * Muestra el nombre del empleado, el puesto y la cantidad de órdenes que ha gestionado.
    * Utiliza GROUP BY y COUNT.

In [38]:
sql = ('SELECT e.nombre_empleado AS EMPLEADO, e.puesto AS PUESTO, count(o.id_empleado) AS ORDENES FROM empleados e JOIN ordenes o ON o.id_empleado = e.id_empleado GROUP BY e.id_empleado')
consultas(sql)

[('Mario Guzmán', 'Cajero', 80),
 ('Irene Ortiz', 'Reponedor', 82),
 ('Adriana Cortés', 'Encargado de sección', 74),
 ('David Lara', 'Gerente', 78),
 ('Laura Aguilar', 'Personal de limpieza', 88),
 ('Verónica Jiménez', 'Panadero', 72),
 ('Miguel Cabrera', 'Carnicero', 73),
 ('Irene Esquivel', 'Pescadero', 75),
 ('Sofía Romero', 'Frutero', 74),
 ('Tomás Moreno', 'Charcutero', 70),
 ('Ana Chávez', 'Jefe de turno', 65),
 ('Cristian Reyes', 'Operador logístico', 59),
 ('Jorge Montes', 'Atención al cliente', 85),
 ('Cristian Cruz', 'Auxiliar de caja', 74),
 ('Ricardo Fernández', 'Encargado de seguridad', 81),
 ('David López', 'Responsable de zona', 82),
 ('Sergio Delgado', 'Jefe de compras', 75),
 ('Camila Paredes', 'Encargada carnicería', 83),
 ('José Flores', 'encargado pescadería', 74),
 ('Mario Serrano', 'Jefe de Limpieza', 68),
 ('Lucía Fuentes', 'Cajero', 74),
 ('Diego Lara', 'Reponedor', 68),
 ('Laura Sánchez', 'Encargado de sección', 63),
 ('Ricardo Ortiz', 'Gerente', 62),
 ('Cristi

* 6.-Ordenes filtradas por fecha y tienda

    * Muestra todas las órdenes que se realizaron en un rango de fechas determinado (ej.: del 1 de enero de 2025 al 31 de enero de 2025) y en una tienda específica.
    * Incluye datos de la tienda y del cliente.

In [39]:
sql = ("SELECT t.id_tienda, t.nombre_tienda AS tienda, o.id_orden AS factura, o.fecha_orden AS fecha, concat(c.first_name,' ',c.last_name) AS cliente FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON o.id_empleado = e.id_empleado JOIN clientes c ON c.id_cliente = o.id_cliente WHERE T.id_tienda = 1 and o.fecha_orden > '2024-01-31' and o.fecha_orden < '2024-03-01' ORDER BY o.fecha_orden;")
consultas(sql)

[(1, 'Tienda Centro', 788, datetime.date(2024, 2, 1), 'Cebolla Méndez'),
 (1, 'Tienda Centro', 790, datetime.date(2024, 2, 1), 'Refresco Montes'),
 (1, 'Tienda Centro', 797, datetime.date(2024, 2, 1), 'Mantequilla Vázquez'),
 (1, 'Tienda Centro', 798, datetime.date(2024, 2, 1), 'Cebolla Fernández'),
 (1,
  'Tienda Centro',
  807,
  datetime.date(2024, 2, 2),
  'Crema hidratante Serrano'),
 (1, 'Tienda Centro', 817, datetime.date(2024, 2, 2), 'Cebolla Martínez'),
 (1, 'Tienda Centro', 824, datetime.date(2024, 2, 2), 'Manzana Carrillo'),
 (1, 'Tienda Centro', 827, datetime.date(2024, 2, 3), 'Velas Navarro'),
 (1, 'Tienda Centro', 831, datetime.date(2024, 2, 3), 'Chuletas Álvarez'),
 (1, 'Tienda Centro', 833, datetime.date(2024, 2, 3), 'Leche Rubio'),
 (1, 'Tienda Centro', 836, datetime.date(2024, 2, 3), 'Refresco Díaz'),
 (1, 'Tienda Centro', 844, datetime.date(2024, 2, 3), 'Carne de res Esquivel'),
 (1, 'Tienda Centro', 853, datetime.date(2024, 2, 3), 'Desodorante Santos'),
 (1, 'Tienda

* 7.-Ranking de productos más vendidos en cada tienda


    * Para cada tienda, muestra los 3 productos más vendidos (en términos de cantidad total).
    * Tendrás que unir tiendas, empleados, ordenes y detalle_orden, además de productos.
    * Usa GROUP BY y ordena por la cantidad sumada (y opcionalmente, un LIMIT 3).

In [40]:
		
sql = ('WITH ranking_productos AS (SELECT t.nombre_tienda, p.nombre_producto, SUM(do.cantidad) AS total_vendido, ROW_NUMBER() OVER (PARTITION BY t.id_tienda ORDER BY SUM(do.cantidad) DESC) AS ranking FROM tiendas t JOIN empleados e ON t.id_tienda = e.id_tienda JOIN ordenes o ON e.id_empleado = o.id_empleado JOIN detalle_orden do ON o.id_orden = do.id_orden JOIN productos p ON do.id_producto = p.id_producto GROUP BY t.id_tienda, t.nombre_tienda, p.id_producto, p.nombre_producto) SELECT nombre_tienda, nombre_producto, total_vendido FROM ranking_productos WHERE ranking <= 3 ORDER BY nombre_tienda, ranking;')
consultas(sql)		
			

[('Central Tiendas', 'Semillas', 1560.0),
 ('Central Tiendas', 'Pilas', 1412.0),
 ('Central Tiendas', 'Chocolate', 1390.0),
 ('Tienda Centro', 'Zanahoria', 1479.0),
 ('Tienda Centro', 'Platos desechables', 1424.0),
 ('Tienda Centro', 'Chocolate', 1389.0),
 ('Tienda Este', 'Desinfectante', 1291.0),
 ('Tienda Este', 'Esponjas', 1265.0),
 ('Tienda Este', 'Chicles', 1234.0),
 ('Tienda Norte', 'Jugo de naranja', 1307.0),
 ('Tienda Norte', 'Carne de res', 1297.0),
 ('Tienda Norte', 'Chocolate', 1287.0),
 ('Tienda Oeste', 'Cerveza', 1578.0),
 ('Tienda Oeste', 'Tierra para macetas', 1410.0),
 ('Tienda Oeste', 'Cerdo', 1397.0),
 ('Tienda Poniente', 'Tierra para macetas', 1440.0),
 ('Tienda Poniente', 'Papas fritas', 1433.0),
 ('Tienda Poniente', 'Uva', 1391.0),
 ('Tienda Sur', 'Semillas', 1422.0),
 ('Tienda Sur', 'Crema hidratante', 1411.0),
 ('Tienda Sur', 'Champú', 1336.0)]